In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Loading the Dataset

In [5]:
data=pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")
data=data.dropna()

In [6]:
data.head()

## EDA

In [7]:
data["HeartDisease"].value_counts()

In [8]:
ax=sns.countplot(x="HeartDisease", data=data)
ax.set_title("Number of samples diagnosed with/without heart disease")

In [9]:
positive_cases=data.loc[data["HeartDisease"]==1]
positive_cases.head()

In [10]:
gender_counts=sns.countplot(x="Sex", data=positive_cases)
gender_counts.set_title("Breakdown of positive diagnoses by gender")

In [11]:
ecg_counts=sns.countplot(x="RestingECG", data=positive_cases)
ecg_counts.set_title("Breakdown of positive diagnoses by resting ECG counts")

In [12]:
exercise_angina_counts=sns.countplot(x="ExerciseAngina", data=positive_cases)
exercise_angina_counts.set_title("Diagnoses resulting from patitents exercising with angina")

In [13]:
chest_pain_counts=sns.countplot(x="ChestPainType", data=positive_cases)
chest_pain_counts.set_title("Breakdown of cases by chest pain types")

## Preprocessing Dataset

In [14]:
data.info()

In [15]:
labels=data["HeartDisease"]
data=data.drop("HeartDisease", axis=1)

In [16]:
from sklearn.preprocessing import OneHotEncoder, Normalizer
from sklearn.compose import ColumnTransformer

In [17]:
ct=ColumnTransformer([("norm1", Normalizer(norm="l1"), [0, 3, 4, 5, 7, 9]), ('encoder', OneHotEncoder(), [1,2,6,8,10])], remainder="passthrough")

In [18]:
X=ct.fit_transform(data)

In [19]:
labels=np.asarray(labels).reshape((-1, 1))

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, y_train, y_test=train_test_split(X, labels, test_size=0.2, random_state=20)

In [22]:
y_train=np.asarray(y_train).reshape((-1,1))
y_test=np.asarray(y_test).reshape((-1,1))

## Creating the Neural Network

In [23]:
import tensorflow as tf
from tensorflow.keras import layers

In [24]:
model=tf.keras.Sequential([
    layers.Dense(32, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")]
)

In [25]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [26]:
model.fit(X_train, y_train, epochs=6, batch_size=16, validation_data=(X_test, y_test))